In [ ]:
!pip install --quiet tensorflow-text
# Clone the entire repo.
%cd /content/
!rm -r -f dl4tm
!git clone git://github.com/Jeilef/DL4TM_Text2Python.git dl4tm
%cd dl4tm
!ls

In [11]:
import numpy as np
import json
import logging
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_text # See https://github.com/tensorflow/hub/issues/463

In [12]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

In [13]:
train_file = open("conala-corpus-v1.1/conala-corpus/conala-train.json", "r")
train_json = json.load(train_file)

print(len(train_json))
print(train_json[0])

2379
{'intent': 'How to convert a list of multiple integers into a single integer?', 'rewritten_intent': "Concatenate elements of a list 'x' of multiple integers to a single integer", 'snippet': 'sum(d * 10 ** i for i, d in enumerate(x[::-1]))', 'question_id': 41067960}


In [14]:
base_sentences = [item["rewritten_intent"] for item in train_json]
codes = [item["snippet"] for item in train_json]

print(base_sentences[0])
print(codes[0])

How to convert a list of multiple integers into a single integer?
sum(d * 10 ** i for i, d in enumerate(x[::-1]))


In [29]:
# Pretrained BERT encoder
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1", trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 128].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 128].

In [30]:
# Transformer Decoder  taken from https://www.tensorflow.org/tutorials/text/transformer
from transformer_util import Decoder
output_tokens = 8000
sample_decoder = Decoder(num_layers=2, d_model=128, num_heads=2,
                         dff=128, target_vocab_size=output_tokens,
                         maximum_position_encoding=5000)

decoder_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
# Use standard keras vectorization for now, maybe replace later
vectorize_layer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=output_tokens,
                                    output_mode='int')
vectorize_layer.adapt(codes)
vectorized_input = vectorize_layer(decoder_input)


decoder_output, attn = sample_decoder(vectorized_input,
                              enc_output=sequence_output,
                              training=False,
                              look_ahead_mask=None,
                              padding_mask=None)
final_layer = tf.keras.layers.Dense(output_tokens)
output = final_layer(decoder_output)


embedding_model = tf.keras.Model((text_input, decoder_input), output)

In [31]:
sentences = tf.constant([train_json[0]["rewritten_intent"]])
snippets = tf.constant([train_json[0]["snippet"]])
result = embedding_model((sentences, snippets))
result

<tf.Tensor: shape=(1, 8, 8000), dtype=float32, numpy=
array([[[-0.3346448 ,  0.03001721, -0.0153175 , ...,  0.35389984,
          0.06700054, -0.13454579],
        [-0.3582388 , -0.00494193, -0.07079062, ...,  0.28480467,
          0.14465441, -0.12122036],
        [-0.3047657 ,  0.02607899, -0.13843787, ...,  0.31949094,
          0.13443479, -0.04309404],
        ...,
        [-0.32030255, -0.02814047, -0.06071242, ...,  0.2847812 ,
          0.18945083, -0.1295053 ],
        [-0.27664897, -0.01990721, -0.02394043, ...,  0.28804892,
          0.15847974, -0.09352209],
        [-0.27658758, -0.03500801, -0.02378963, ...,  0.2544594 ,
          0.17877574, -0.09700234]]], dtype=float32)>

In [9]:
# TODO: Preprocess code samples
# TODO: Set up masking
# TODO: Run training